# Assignment 5: Document Retrieval and Evaluation (Part 3)
## Dreamy Pujara - 202211005
### IT550: Information Retrieval

## Pyterrier Installation and Declaration

In [ ]:
!pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ..

In [ ]:
import pyterrier as pt
pt.init()

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



## MSMACRO DataSet

In [ ]:
dataset = pt.get_dataset("trec-deep-learning-docs")

## Indexing

In [ ]:
indexer = pt.TRECCollectionIndexer("./index")
#this downloads the file msmarco-docs.trec.gz

In [ ]:
indexref = indexer.index(dataset.get_corpus())
index = pt.IndexFactory.of(indexref)

msmarco-docs.trec.gz:   0%|          | 0.00/7.92G [00:00<?, ?iB/s]

## BatchRetrive WModels

- BatchRetrieve is one of the most commonly used PyTerrier objects. It represents a retrieval transformation, in which queries are mapped to retrieved documents. BatchRetrieve uses a pre-existing Terrier index data structure, typically saved on disk.

- BM25	:This class implements the Okapi BM25 weighting model.
- BM25F	:A convenience subclass of PerFieldNormWeightingModel setup to do specifically BM25F
- PL2	This class implements the PL2 weighting model.
-PL2F	:A convenience subclass of PerFieldNormWeightingModel setup to do specifically PL2F.
- TF_IDF	:This class implements the TF_IDF weighting model.

In [ ]:
TFIDF = pt.BatchRetrieve(index, wmodel="TFIDF")

In [ ]:
BM25 = pt.BatchRetrieve(index, wmodel="BM25")

In [ ]:
PL2 = pt.BatchRetrieve(index, wmodel="PL2")

In [ ]:
BM25F=pt.BatchRetrieve(index,wmodel="BM25F")

In [ ]:
PL2F=pt.BatchRetrieve(index,wmodel="PL2F")

### Query & Topics

In [ ]:
qrels = dataset.get_qrels("test")

2019qrels-docs.txt:   0%|          | 0.00/331k [00:00<?, ?iB/s]

In [ ]:
topics=dataset.get_topics("test")

msmarco-test2019-queries.tsv.gz:   0%|          | 0.00/4.18k [00:00<?, ?iB/s]

## Experiments

In [ ]:
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
retr.setControl("wmodel", "TF_IDF")
retr.setControls({"wmodel": "TF_IDF"})

res=retr.transform(topics)

In [ ]:
eval = pt.Utils.evaluate(res,qrels)
eval

{'map': 0.3332053881157273, 'ndcg': 0.5994880938468127}

## TFIDF,BM25,PL2 with all supported measures

- PyTerrier aims to make it easy to conduct an information retrieval experiment, namely, to run a transformer pipeline over a set of queries, and evaluating the outcome using standard information retrieval evaluation metrics based on known relevant documents

In [ ]:
import ir_measures
from ir_measures import * # imports all supported measures, e.g., AP, nDCG, RR, P

In [ ]:
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")

pt.Experiment([tf_idf, bm25, pl2], topics, qrels, eval_metrics=[RR@10,RR@100,AP@10,AP@100,nDCG@10, nDCG@100])

,name,RR@10,RR@100,AP@10,AP@100,nDCG@10
0,BR(TF_IDF),0.864987,0.866044,0.113309,0.244909,0.524194
1,BR(BM25),0.874123,0.874123,0.114305,0.248334,0.538802
2,BR(PL2),0.841085,0.842746,0.112509,0.248535,0.542381


### DPH

In [ ]:
props = {
  'indexer.meta.reverse.keys':'docno',
  'termpipelines' : '',
}

indexer = pt.TRECCollectionIndexer("./index")
indexer.setProperties(**props)
indexref = indexer.index(dataset.get_corpus())


msmarco-docs.trec.gz:   0%|          | 0.00/7.92G [00:00<?, ?iB/s]

<ipython-input-7-3f2140bdbe23>:8: DeprecationWarning: Setting of termpipelines property directly is deprecated
  indexref = indexer.index(dataset.get_corpus())


In [ ]:
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 3213835
Number of terms: 17470544
Number of postings: 1234274977
Number of fields: 0
Number of tokens: 3667907097
Field names: []
Positions:   false



In [ ]:
DPH_br = pt.BatchRetrieve(index, wmodel="DPH", properties={"termpipelines": ""}, verbose=True) % 100

In [ ]:

pt.Experiment([DPH_br], topics, qrels, eval_metrics=[RR@10,RR@100,AP@10,AP@100,nDCG@10, nDCG@10])

BR(DPH):   0%|          | 0/200 [00:00<?, ?q/s]

,name,RR@10,RR@100,AP@10,AP@100,nDCG@10
0,"RankCutoff(BR(DPH), 100)",0.843632,0.844527,0.105913,0.225402,0.513208


### BM25F, PL2F

In [ ]:
fields=['title','body']
indexer_with_fields= pt.TRECCollectionIndexer("./index_with_fields")

In [ ]:
pt.ApplicationSetup.setProperty("indexer.meta.forward.keys",",".join(fields))
pt.ApplicationSetup.setProperty("indexer.meta.reverse.keys",",".join(fields))
pt.ApplicationSetup.setProperty("FieldTags.process",",".join(fields))
pt.ApplicationSetup.setProperty("TrecDocTags.doctag","DOC")
pt.ApplicationSetup.setProperty("TrecDocTags.idtag","DOCNO")
pt.ApplicationSetup.setProperty("TrecDocTags.skip","DOCHDR")

In [ ]:
indexref = indexer_with_fields.index(dataset.get_corpus())
index = pt.IndexFactory.of(indexref)

msmarco-docs.trec.gz:   0%|          | 0.00/7.92G [00:00<?, ?iB/s]

In [ ]:
bm25f = pt.BatchRetrieve(index, wmodel="BM25F")
pl2f = pt.BatchRetrieve(index, wmodel="PL2F")

pt.Experiment([bm25f, pl2f], topics, qrels, eval_metrics=[RR@10,RR@100,AP@10,AP@100,nDCG@10, nDCG@100],names=["BM25F","PL2F"])

,name,RR@10,RR@100,AP@10,AP@100,nDCG@10,nDCG@100
0,BM25F,0.011628,0.011878,0.023256,0.023259,0.018011,0.018193
1,PL2F,0.011628,0.011878,0.023256,0.023259,0.018011,0.018197
